In [1]:
from model1 import bitext
from model1 import Model1 as m1
from model1 import*
from collections import defaultdict
from bleu import*
%load_ext autoreload
%autoreload 2

#### program has been submitted separately as a .py file

#### sample terminal input/output

In [2]:
%%time
IBM_model = None
with open("hansards.36.ca.e.tok", "r") as target:
    with open("hansards.36.ca.f.tok", "r") as source:
        IBM_model = m1(source, target)

CPU times: user 26.8 s, sys: 249 ms, total: 27 s
Wall time: 27.2 s


In [3]:
table = IBM_model.train(5)

##### Best translation for the 50 French words

In [5]:
fwords = []
with open("fwords.txt", "r") as fw:
    for w in fw:
        fwords.append(w.strip())
translate(table, fwords)

AFFAIRES AFFAIRS
AMENDEMENT AMENDMENT
AUTOCHTONES ABORIGINAL
CHAMBRE HOUSE
CHEF LEADER
COMITÉ COMMITTEE
COMMERCE TRADE
COMMUNES COMMONS
DROITS RIGHTS
DÉCLARATION STATEMENT
DÉFENSE DEFENCE
DÉVELOPPEMENT DEVELOPMENT
ENFANTS CHILDREN
FINANCES FINANCE
FÉDÉRAL FEDERAL
GENS PEOPLE
GOUVERNEMENT GOVERNMENT
GUERRE WAR
HISTOIRE HISTORY
INTERNATIONALE INTERNATIONAL
JUGE JUSTICE
MINISTÈRE DEPARTMENT
MONDE WORLD
NATIONALE NATIONAL
PARLEMENT PARLIAMENT
PAROLE SPEAK
PARTIE PART
PREMIER PRIME
PREMIÈRE FIRST
PROGRAMME PROGRAM
PROJET BILL
PROVINCE PROVINCE
PRÉSIDENT SPEAKER
QUÉBEC QUEBEC
RAISON REASON
RAPPORT REPORT
RESPONSABILITÉ RESPONSIBILITY
RÉGIME PLAN
RÉGION REGION
RÉPONSE ANSWER
SECTEUR SECTOR
SERVICES SERVICES
SOCIÉTÉ SOCIETY
SÉCURITÉ SECURITY
SÉNAT SENATE
SÉNATEUR SENATOR
TRAVAIL WORK
ÉGALEMENT ALSO
ÉTATS-UNIS STATES
ÉTUDE STUDY


##### Approach/bugs/fixes

I was initially having some problem with the data structure and my program took forever to compute. After having a discussion with you, I switched to using `deafultdicts` and only made entries for the sources/target words which are in the same bilingual parallel sentences. This greatly reduced the memory size and increased the speed of my program. 

In [11]:
refs = []
hyps1 = []
hyps2 = []
with open("e.tox.tok", "r") as ref:
    for line in ref:
        refs.append(line)
with open("gtranslate.tok", "r") as hyp:
    for line in hyp:
        hyps1.append(line)
with open("systran.tok", "r") as hyp:
    for line in hyp:
        hyps2.append(line)
        

In [14]:
for i in range(len(hyps)):
    print("gtranslate score :", BLEU(hyps1[i],refs[i]), "systran score :", BLEU(hyps2[i],refs[i]))

gtranslate score : 0.5719246937999647 systran score : 0.3922779324734805
gtranslate score : 0.28568042338597716 systran score : 0.28956365673702783
gtranslate score : 0.427639889086126 systran score : 0.4787898732732043
gtranslate score : 0.4411629359322707 systran score : 0.3492382342001821
gtranslate score : 0.2975990331686359 systran score : 0.25666772456849113
gtranslate score : 0.9048374180359595 systran score : 0.8931539818068694
gtranslate score : 0.2316809767568795 systran score : 0.4469948199291566
gtranslate score : 0.40016016019225 systran score : 0.40652204338608683
gtranslate score : 0.5842896293786666 systran score : 0.3815474751830903
gtranslate score : 0.6999271023161167 systran score : 0.36939832906091624


##### Discussion on BLEU score results 

Overall I think gtranslate seems to be performing better than systran. The high score phrases were mostly noun to noun translations. But some sentences even though made sence and was conceptually correct surprisingly received a very low BLEU score. Like in this example where the reference was `MANY WILL LOSE THEIR RIGHT TO A PENSION IN THEIR OWN NAME BECAUSE OF THEIR HUSBAND 'S INCOME , WHILE MANY OTHERS WILL SEE THEIR PENSIONS REDUCED` and the sentence generated by gtranslate was `MANY WILL LOSE THEIR RIGHT TO A PENSION IN THEIR OWN NAME BECAUSE OF THEIR HUSBANDS ' INCOME WHILE OTHERS WILL REDUCE THE AMOUNT OF THEIR PENSION .`

In [15]:
for i in range(len(hyps)):
    print("gtranslate score :", BLEU(hyps1[i],refs[i]), refs[i], hyps1[i])

gtranslate score : 0.5719246937999647 MANY WILL LOSE THEIR RIGHT TO A PENSION IN THEIR OWN NAME BECAUSE OF THEIR HUSBAND 'S INCOME , WHILE MANY OTHERS WILL SEE THEIR PENSIONS REDUCED .
 MANY WILL LOSE THEIR RIGHT TO A PENSION IN THEIR OWN NAME BECAUSE OF THEIR HUSBANDS ' INCOME WHILE OTHERS WILL REDUCE THE AMOUNT OF THEIR PENSION .

gtranslate score : 0.28568042338597716 THE COMBINED PENSION WILL CREATE A PROBLEM FOR SENIOR WOMEN BECAUSE IT WILL ELIMINATE ALL THAT SOME OF THEM HAVE TO CALL THEIR OWN , TO BUY A GIFT FOR A GRANDCHILD PERHAPS , OR TO SPEND ON THEMSELVES .
 THE COMBINED PENSION WILL CREATE A PROBLEM FOR OLDER WOMEN BECAUSE IT ELIMINATES EVERYTHING THAT CAN CLAIM SOME OF THEM FOR THEIR PERSONAL PURCHASES OR , FOR EXAMPLE , TO BUY GIFTS FOR THEIR GRANDCHILDREN .

gtranslate score : 0.427639889086126 HAS THE GOVERNMENT DONE ANY CALCULATIONS CONCERNING HOW THIS NEW MEASURE WILL AFFECT WOMEN ?
 THE GOVERNMENT HE CALCULATED THE IMPACT OF THE NEW BENEFIT WOMEN ?

gtranslate score

##### Approach/bugs/fixes

I pretty much followed the equations given on the paper. I had a list of initialized which could hold all the modified precisions. My function then loops over different ngrams and computes the sum. I ran into two bugs which I was able to fix with some debugging. I was initially getting zero BLEU scores for all the translations. I then printed my counters to see if the ngram counts are being changed. The counts weren't changing because, I was calling `list()` on the input sentence which passed characters to the ngram function instead of words. I then fixed it by using the `split()` method. Another bug I had was, I was getting negative scores for some reason and I then instantly noticed that I was directly multiplying the penalty with sum instead of multiplying with the exponential. After I fixed it, BLEU scores seemed reasonable. 